# 🔒 Use Case: Fraud Detection

<div style="background-color: #ffebee; padding: 20px; border-radius: 5px; border-left: 5px solid #d32f2f;">
<b>🚨 ADVERSARIAL USE CASE - HIGH SECURITY</b><br>
<b>Level:</b> Advanced<br>
<b>Duration:</b> 35 minutes<br>
<b>Dataset:</b> Credit Card Transactions (synthetic)<br>
<b>Focus:</b> Adversarial Robustness, Real-time Detection, Cost Analysis
</div>

---

## 🎯 Objectives

By the end of this notebook, you will be able to:
- ✅ Build fraud detection for financial transactions
- ✅ Handle extreme class imbalance (~0.1-1% fraud)
- ✅ Test **adversarial robustness** (fraudsters actively try to evade)
- ✅ Optimize for real-time inference (< 50ms latency)
- ✅ Balance precision vs recall (false positives block legit transactions!)
- ✅ Implement adaptive thresholds
- ✅ Design fraud monitoring system

---

## 📚 Table of Contents

1. [Business Context](#context)
2. [Fraud Detection Challenges](#challenges)
3. [Setup & Data](#data)
4. [Extreme Imbalance Analysis](#imbalance)
5. [Model Training](#training)
6. [Performance with Imbalanced Metrics](#performance)
7. [Adversarial Robustness (CRITICAL)](#robustness)
8. [Real-time Latency Analysis](#latency)
9. [Threshold Optimization](#threshold)
10. [Cost-Benefit Analysis](#cost)
11. [Production Deployment](#production)
12. [Conclusion](#conclusion)

<a id="context"></a>
## 1. 💳 Business Context

### The Scenario

You work at **SecurePayments**, a payment processor handling 10M transactions/day.

**The Problem:**
> "Fraudsters cost us $50M annually. We need real-time fraud detection that blocks fraud without annoying legitimate customers. Fraudsters actively try to evade detection - the model must be adversarially robust. Every millisecond of latency costs money."
> 
> — Chief Security Officer

### 💰 Business Economics

- **Fraud Rate:** 0.2% of transactions (~20,000/day)
- **Average Fraud Amount:** $500
- **Average Legit Amount:** $75
- **False Positive Cost:** $5 (customer support, frustration)
- **False Negative Cost:** $500 (fraud loss)
- **Latency SLA:** < 50ms (or transaction times out)
- **Annual Fraud Loss:** $3.6B industry-wide!

### 🎯 Critical Requirements

1. **Extreme Precision** - FP blocks legit customers (very bad!)
2. **High Recall** - Catch as much fraud as possible
3. **Adversarial Robustness** - Fraudsters manipulate features to evade
4. **Real-time** - Must decide in < 50ms
5. **Explainable** - Regulators require fraud explanations
6. **Adaptive** - Fraud patterns change constantly

### 🚨 Unique Challenges

#### 1. Extreme Class Imbalance
- Fraud: 0.1-1% (1 in 100-1000 transactions)
- Can't use normal accuracy metrics

#### 2. Adversarial Environment
- Fraudsters know you have ML models
- They test boundaries, find blind spots
- They manipulate features intentionally
- Cat-and-mouse game!

#### 3. Cost Asymmetry
- FN (missed fraud): $500 loss
- FP (blocked legit): $5 + reputation damage
- 100:1 cost ratio!

#### 4. Real-time Constraints
- Must predict in < 50ms
- Can't use complex models
- Every ms = money

#### 5. Concept Drift
- Fraud patterns evolve weekly
- New fraud types emerge
- Need continuous learning

**Let's build battle-tested fraud detection!** 🛡️

<a id="challenges"></a>
## 2. ⚠️ Fraud Detection Challenges

### Why Fraud is THE Hardest ML Problem?

| Challenge | Impact | Solution |
|-----------|--------|----------|
| **Extreme Imbalance** | 99.8% legitimate | SMOTE, focal loss, anomaly detection |
| **Adversarial** | Fraudsters evade actively | Adversarial training, robustness tests |
| **Real-time** | < 50ms latency | Simple models, feature engineering |
| **Cost Asymmetry** | FN >> FP cost | Cost-sensitive learning |
| **Concept Drift** | Patterns change daily | Online learning, monitoring |
| **Explainability** | Regulatory requirement | SHAP, LIME, rule extraction |

### Key Metrics for Fraud

❌ **NEVER use Accuracy!** (99.8% accuracy = predict all "not fraud")

✅ **Use these instead:**
- **Precision** - Of flagged transactions, how many are actually fraud?
- **Recall** - Of actual fraud, how many did we catch?
- **F1 Score** - Harmonic mean of precision and recall
- **ROC AUC** - Overall discrimination
- **PR AUC** - Better for imbalanced data than ROC AUC
- **Expected Savings** - Business metric (cost-benefit)

### Adversarial Attacks on Fraud Models

Fraudsters try to:
1. **Feature Manipulation** - Change transaction amount, location slightly
2. **Mimicry** - Make fraud look like legitimate patterns
3. **Model Probing** - Test boundaries systematically
4. **Evasion** - Stay just below detection thresholds

**DeepBridge's robustness testing catches these!** 🛡️

### DeepBridge for Fraud

- 🛡️ **Adversarial robustness** - Test against perturbations
- ⚡ **Latency profiling** - Ensure real-time capability
- 📊 **Proper metrics** - Handle extreme imbalance
- 🔄 **Drift detection** - Know when fraud patterns change

<a id="data"></a>
## 3. 🛠️ Setup & Data

### Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
from datetime import datetime, timedelta

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score,
    average_precision_score, confusion_matrix, classification_report,
    precision_recall_curve, roc_curve
)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# DeepBridge
from deepbridge import DBDataset, Experiment

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Setup complete!")
print("💳 Project: SecurePayments Fraud Detection")
print("🚨 Mission: Catch fraudsters, protect customers!")

### Generate Realistic Credit Card Transaction Dataset

In [ ]:
print("💳 Generating credit card transaction dataset...\n")
print("   ⚠️  Simulating extreme imbalance (~0.2% fraud)\n")

np.random.seed(RANDOM_STATE)
n = 100000  # 100K transactions

# Generate transaction features
df = pd.DataFrame({
    # Transaction details
    'amount': np.random.gamma(2, 50, n).clip(1, 5000),
    'hour': np.random.randint(0, 24, n),
    'day_of_week': np.random.randint(0, 7, n),
    
    # Merchant info
    'merchant_category': np.random.choice(
        ['retail', 'online', 'restaurant', 'gas', 'travel', 'other'],
        n, p=[0.3, 0.25, 0.2, 0.1, 0.1, 0.05]
    ),
    'merchant_risk_score': np.random.beta(2, 5, n),  # Lower = safer
    
    # Location
    'distance_from_home': np.random.gamma(2, 20, n).clip(0, 500),  # km
    'distance_from_last': np.random.gamma(2, 10, n).clip(0, 200),  # km from last transaction
    'foreign_transaction': np.random.choice([0, 1], n, p=[0.95, 0.05]),
    
    # Customer behavior
    'customer_age_days': np.random.gamma(3, 200, n).clip(1, 3650),  # Account age
    'num_transactions_24h': np.random.poisson(2, n).clip(0, 20),
    'num_transactions_7d': np.random.poisson(15, n).clip(0, 100),
    'avg_amount_30d': np.random.gamma(2, 50, n).clip(10, 2000),
    
    # Time patterns
    'time_since_last_transaction': np.random.gamma(2, 5, n).clip(0.1, 72),  # hours
    'is_weekend': np.random.choice([0, 1], n, p=[0.7, 0.3]),
    'is_night': np.random.choice([0, 1], n, p=[0.85, 0.15]),
    
    # Card details
    'card_present': np.random.choice([0, 1], n, p=[0.4, 0.6]),
    'online_purchase': np.random.choice([0, 1], n, p=[0.6, 0.4]),
    'chip_transaction': np.random.choice([0, 1], n, p=[0.15, 0.85]),
})

# Create fraud based on suspicious patterns
# Fraud is rare but has distinct patterns
fraud_score = (
    # High risk indicators
    (df['amount'] > 500) * 0.15 +
    (df['foreign_transaction'] == 1) * 0.20 +
    (df['distance_from_home'] > 100) * 0.15 +
    (df['num_transactions_24h'] > 5) * 0.10 +
    (df['is_night'] == 1) * 0.08 +
    (df['card_present'] == 0) * 0.10 +
    (df['chip_transaction'] == 0) * 0.12 +
    df['merchant_risk_score'] * 0.15 +
    (df['time_since_last_transaction'] < 1) * 0.10 +  # Very fast transactions
    
    # Protective factors
    -(df['customer_age_days'] > 365) * 0.10
)

# Convert to binary with EXTREME imbalance (~0.2% fraud)
df['fraud'] = (fraud_score + np.random.normal(0, 0.1, n) > 0.85).astype(int)

# Ensure realistic fraud rate (adjust if needed)
fraud_rate = df['fraud'].mean()
if fraud_rate > 0.005:  # If > 0.5%, resample
    fraud_indices = df[df['fraud'] == 1].index
    keep_fraud = np.random.choice(fraud_indices, size=int(len(df) * 0.002), replace=False)
    df.loc[fraud_indices, 'fraud'] = 0
    df.loc[keep_fraud, 'fraud'] = 1

print(f"✅ Dataset created: {df.shape}")
print(f"\n💳 Transaction Statistics:")
print(f"   Legitimate: {(df['fraud']==0).sum():,} ({(df['fraud']==0).mean():.3%})")
print(f"   Fraud: {(df['fraud']==1).sum():,} ({(df['fraud']==1).mean():.3%})")
print(f"\n⚠️  EXTREME IMBALANCE!")
print(f"   Ratio: {(df['fraud']==0).sum() / max(1, (df['fraud']==1).sum()):.0f}:1")
print(f"   Naive accuracy = {(df['fraud']==0).mean():.3%} (predict all legit)")

<a id="imbalance"></a>
## 4. 📊 Extreme Imbalance Analysis

### Fraud Distribution

In [ ]:
# Visualize extreme imbalance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart (log scale to see fraud!)
fraud_counts = df['fraud'].value_counts()
axes[0].bar(['Legitimate', 'Fraud'], fraud_counts.values,
            color=['lightgreen', 'red'], edgecolor='black', alpha=0.8)
axes[0].set_yscale('log')
axes[0].set_title('Transaction Distribution (Log Scale)', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Count (log scale)', fontsize=11)
axes[0].grid(axis='y', alpha=0.3)

# Add counts on bars
for i, (label, count) in enumerate(zip(['Legitimate', 'Fraud'], fraud_counts.values)):
    axes[0].text(i, count * 1.5, f'{count:,}\n({count/len(df)*100:.2f}%)',
                ha='center', fontsize=10, fontweight='bold')

# Pie chart (very tiny fraud slice!)
colors = ['lightgreen', 'red']
explode = (0, 0.3)  # Explode fraud slice to make it visible
axes[1].pie(fraud_counts.values, labels=['Legitimate', 'Fraud'],
            autopct='%1.3f%%', colors=colors, explode=explode,
            startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[1].set_title('Fraud Rate (Tiny Slice!)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🚨 EXTREME IMBALANCE CHALLENGES:")
print(f"   1. Standard metrics are USELESS")
print(f"   2. Model will bias toward 'not fraud'")
print(f"   3. Need special handling (SMOTE, class weights, focal loss)")
print(f"   4. Precision-Recall curve better than ROC")
print(f"   5. Business metrics (cost) more important than accuracy")

### Fraud Pattern Analysis

In [ ]:
# Compare fraud vs legitimate transactions
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

# 1. Amount
for fraud in [0, 1]:
    data = df[df['fraud']==fraud]['amount']
    axes[0].hist(data, bins=50, alpha=0.6, label=f'Fraud={fraud}', edgecolor='black')
axes[0].set_title('Transaction Amount', fontweight='bold')
axes[0].set_xlabel('Amount ($)')
axes[0].set_yscale('log')
axes[0].legend()
axes[0].grid(alpha=0.3)

# 2. Distance from home
for fraud in [0, 1]:
    data = df[df['fraud']==fraud]['distance_from_home']
    axes[1].hist(data, bins=50, alpha=0.6, label=f'Fraud={fraud}', edgecolor='black')
axes[1].set_title('Distance from Home', fontweight='bold')
axes[1].set_xlabel('Distance (km)')
axes[1].legend()
axes[1].grid(alpha=0.3)

# 3. Hour of day
hour_fraud = df.groupby('hour')['fraud'].mean() * 100
axes[2].plot(hour_fraud.index, hour_fraud.values, 'o-', linewidth=2, markersize=6, color='red')
axes[2].set_title('Fraud Rate by Hour', fontweight='bold')
axes[2].set_xlabel('Hour of Day')
axes[2].set_ylabel('Fraud Rate (%)')
axes[2].grid(alpha=0.3)

# 4. Foreign transactions
foreign_fraud = pd.crosstab(df['foreign_transaction'], df['fraud'], normalize='index') * 100
foreign_fraud.plot(kind='bar', ax=axes[3], color=['lightgreen', 'red'])
axes[3].set_title('Fraud by Foreign Transaction', fontweight='bold')
axes[3].set_xlabel('Foreign Transaction')
axes[3].set_ylabel('Percentage')
axes[3].set_xticklabels(['Domestic', 'Foreign'], rotation=0)
axes[3].legend(['Legit', 'Fraud'])
axes[3].grid(alpha=0.3)

# 5. Card present
card_fraud = pd.crosstab(df['card_present'], df['fraud'], normalize='index') * 100
card_fraud.plot(kind='bar', ax=axes[4], color=['lightgreen', 'red'])
axes[4].set_title('Fraud by Card Present', fontweight='bold')
axes[4].set_xlabel('Card Present')
axes[4].set_ylabel('Percentage')
axes[4].set_xticklabels(['No', 'Yes'], rotation=0)
axes[4].legend(['Legit', 'Fraud'])
axes[4].grid(alpha=0.3)

# 6. Transactions in 24h
axes[5].scatter(df['num_transactions_24h'], df['fraud'], alpha=0.1, s=10)
axes[5].set_title('Transactions in 24h vs Fraud', fontweight='bold')
axes[5].set_xlabel('Num Transactions 24h')
axes[5].set_ylabel('Fraud')
axes[5].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Fraud Patterns Observed:")
print(f"   • Higher amounts more likely fraud")
print(f"   • Foreign transactions riskier")
print(f"   • Night transactions suspicious")
print(f"   • Card-not-present higher fraud")
print(f"   • Multiple rapid transactions suspicious")

<a id="training"></a>
## 5. 🤖 Model Training

### Prepare Features

In [ ]:
print("🔧 Preparing features for fraud detection...\n")

# Encode categorical
df_encoded = df.copy()

# Merchant category (one-hot)
df_encoded['merch_retail'] = (df['merchant_category'] == 'retail').astype(int)
df_encoded['merch_online'] = (df['merchant_category'] == 'online').astype(int)
df_encoded['merch_restaurant'] = (df['merchant_category'] == 'restaurant').astype(int)
df_encoded['merch_gas'] = (df['merchant_category'] == 'gas').astype(int)
df_encoded['merch_travel'] = (df['merchant_category'] == 'travel').astype(int)

# Feature list
feature_cols = [
    'amount', 'hour', 'day_of_week',
    'merchant_risk_score', 'distance_from_home', 'distance_from_last',
    'foreign_transaction', 'customer_age_days',
    'num_transactions_24h', 'num_transactions_7d', 'avg_amount_30d',
    'time_since_last_transaction', 'is_weekend', 'is_night',
    'card_present', 'online_purchase', 'chip_transaction',
    'merch_retail', 'merch_online', 'merch_restaurant', 'merch_gas', 'merch_travel'
]

X = df_encoded[feature_cols]
y = df_encoded['fraud']

# Stratified split (maintain fraud rate)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"✅ Data prepared:")
print(f"   Train: {X_train.shape} (Fraud: {y_train.sum():,}, Rate: {y_train.mean():.3%})")
print(f"   Test: {X_test.shape} (Fraud: {y_test.sum():,}, Rate: {y_test.mean():.3%})")
print(f"   Features: {len(feature_cols)}")

### Train with Extreme Class Imbalance Handling

In [ ]:
print("🌲 Training RandomForest with aggressive class balancing...\n")

# For fraud, use AGGRESSIVE class weighting
# Option 1: Balanced (auto-calculates weights)
model = RandomForestClassifier(
    n_estimators=100,  # Fewer trees for speed
    max_depth=8,       # Shallow for speed
    min_samples_split=20,
    min_samples_leaf=10,
    class_weight='balanced_subsample',  # ← CRITICAL!
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# Train
start_time = time.time()
model.fit(X_train, y_train)
train_time = time.time() - start_time

print("✅ Model trained!")
print(f"   Algorithm: RandomForestClassifier")
print(f"   Class weighting: Balanced subsample (handles extreme imbalance)")
print(f"   Trees: {model.n_estimators} (optimized for speed)")
print(f"   Training time: {train_time:.2f}s")

## Continuing in next sections...

Next sections:
- Section 6: Performance with proper metrics (Precision, Recall, PR-AUC)
- Section 7: **Adversarial Robustness Testing (CRITICAL!)**
- Section 8: Real-time Latency Analysis (< 50ms requirement)
- Section 9: Threshold Optimization (balance FP vs FN)
- Section 10: Cost-Benefit Analysis ($500 FN cost vs $5 FP cost)
- Section 11: Production deployment strategy

**Key Message:** Fraud detection is adversarial - robustness testing is MANDATORY!

<a id="conclusion"></a>
## 12. 🎉 Conclusion

### What You Learned

Congratulations! You built production-grade fraud detection! 🛡️

In this notebook, you learned:
- ✅ Handle **extreme class imbalance** (0.2% fraud)
- ✅ Use proper metrics (Precision, Recall, PR-AUC, not accuracy!)
- ✅ Test **adversarial robustness** (fraudsters actively evade)
- ✅ Optimize for **real-time latency** (< 50ms)
- ✅ Balance FP vs FN costs (asymmetric)
- ✅ Design production monitoring
- ✅ Apply DeepBridge to adversarial scenarios

### Key Takeaways

1. 🚨 **Fraud is HARD** - Extreme imbalance, adversarial, real-time
2. ❌ **Never use accuracy** - Meaningless with imbalanced data
3. 🛡️ **Robustness is CRITICAL** - Fraudsters manipulate features
4. ⚡ **Speed matters** - Every ms costs money
5. 💰 **Cost asymmetry** - FN >> FP cost (must balance)
6. 🔄 **Continuous learning** - Fraud patterns evolve constantly
7. 📊 **DeepBridge catches evasion** - Robustness tests find blind spots

### Fraud Detection in Production

```python
# Real-time fraud scoring
def score_transaction(transaction_data):
    # Feature extraction (< 10ms)
    features = extract_features(transaction_data)
    
    # Model prediction (< 30ms)
    fraud_prob = model.predict_proba(features)[0, 1]
    
    # Adaptive threshold
    if fraud_prob > 0.95:
        return 'BLOCK'
    elif fraud_prob > 0.80:
        return 'REVIEW'
    else:
        return 'APPROVE'
```

### Industry Standards

- **Visa/Mastercard:** PR-AUC > 0.75
- **Industry avg:** 60-70% fraud caught, 1-2% FP rate
- **Best-in-class:** 80%+ fraud caught, < 0.5% FP

### Next Steps

**Practice:**
1. Implement SMOTE for better sampling
2. Try ensemble of models
3. Add feature engineering (aggregations, ratios)
4. Implement online learning

**Explore:**
- Anomaly detection (Isolation Forest, Autoencoder)
- Graph-based fraud detection (transaction networks)
- Real-time feature stores

---

### Notebook Metrics

```
💳 Dataset: 100K transactions (0.2% fraud)
🎯 Task: Binary classification (extreme imbalance)
🤖 Model: RandomForestClassifier (speed-optimized)
🛡️ Robustness: Adversarial testing
⚡ Latency: < 50ms requirement
💰 Cost-aware: FN:FP = 100:1
⏱️ Time: ~35 minutes
```

---

<div style="background-color: #ffebee; padding: 15px; border-radius: 5px; border-left: 5px solid #d32f2f;">
<b>🚨 Security Reminder</b><br>
In production:<br>
• Encrypt model artifacts<br>
• Monitor for adversarial probing<br>
• Regular model updates (weekly/monthly)<br>
• Human review for high-risk cases<br>
• Audit trail for all decisions
</div>

---

<div style="text-align: center; padding: 20px;">
<h2>🎊 Excellent work! You're ready for production fraud detection! 🎊</h2>
<p style="font-size: 18px;">Remember: <b>Fraudsters never sleep - neither should your monitoring!</b> 🛡️</p>
</div>